In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,plot_tree as plot_decision_tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import warnings
import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Data/invoice_level_data_full_1.csv")


In [ ]:
import datetime
df['disbursal_date'] = pd.to_datetime(df['disbursal_date'])
target_date = datetime.datetime(2024, 2, 1)
df = df[df['disbursal_date']>target_date]
print(len(df))

138177


In [ ]:
from datetime import datetime
df['overdue_date'] = pd.to_datetime(df['overdue_date'])
df['repaid_date'] = pd.to_datetime(df['repaid_date'])
df['disbursal_date'] = pd.to_datetime(df['disbursal_date'])
# Get today's date
today = datetime.today()

# Filter rows
df = df[(df['repaid_date'].notnull()) | (df['overdue_date'] <= today)]

df['dd_tenor'] = (df['overdue_date']-df['disbursal_date']).dt.days

df['repaid_date'] = pd.to_datetime(df['repaid_date'])
# Define a function to calculate the difference in days
def calculate_days(row):
    if pd.isnull(row['repaid_date']):
        return 10000
    else:
        return (row['repaid_date'] - row['overdue_date']).days

def determine_rpay(row):
    if pd.isnull(row['repaid_date']):
        return 120000
    else:
        return (row['repaid_date'] - row['disbursal_date']).days

df['repayment_days'] = df.apply(determine_rpay,axis=1)
df['cashback_date'] = pd.to_datetime(df['cashback_date'])

# def determine_offer(row):
#     if pd.isnull(row['repaid_date']):
#         return 0
#     elif row['cashback_date'] > row['repaid_date']:
#         return 1
#     else:
#         return 0

# df['offer'] = df.apply(determine_offer, axis=1)

def determine_default(row):
    if pd.isnull(row['repaid_date']):
        return 0
    elif row['overdue_date'] > row['repaid_date']:
        return 1
    else:
        return 0
df['default'] = df.apply(determine_default, axis=1)



In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138177 entries, 3 to 1519857
Data columns (total 53 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   Unnamed: 0                            138177 non-null  int64         
 1   facility_id                           138177 non-null  int64         
 2   case_id                               138177 non-null  float64       
 3   customer_id                           138177 non-null  object        
 4   buyer_id                              138177 non-null  float64       
 5   buyer_name                            138177 non-null  object        
 6   buyer_gstin                           138177 non-null  object        
 7   seller_id                             138177 non-null  float64       
 8   seller_name                           138177 non-null  object        
 9   seller_gstin                          138177 non-null  object  

In [ ]:
def clean_df(df):
    columns_to_drop = [
        'Unnamed: 0', 'facility_id', 'case_id', 'customer_id', 'buyer_id', 'buyer_name', 'buyer_gstin', 'seller_id',
        'seller_name', 'seller_gstin', 'anchor_id', 'anchor_gstin', 'anchor_name', 'constitution', 'first_disbursal_date',
        'is_term_loan', 'rff_id', 'invoice_no', 'invoice_date', 'cashback_date', 'overdue_date', 'repaid_date', 'dwd_date',
        'pincode', 'asm_v2', 'zonal_head_v2', 'branch_head_v2', 'state_head_v2', 'progcap_arp_before_disbursal',
        'cibil_score', 'cibil_pull_date', 'churn'
    ]

    # Find which columns are actually present in the DataFrame
    columns_to_drop_present = [col for col in columns_to_drop if col in df.columns]

    return df.drop(columns=columns_to_drop_present)

df = clean_df(df)

In [ ]:
df['dd_tenor'] = pd.to_numeric(df['dd_tenor'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138177 entries, 3 to 1519857
Data columns (total 21 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   industry_name                         138177 non-null  object        
 1   program_type                          138177 non-null  object        
 2   buyer_city                            137267 non-null  object        
 3   state_name                            138177 non-null  object        
 4   invoice_amount                        138177 non-null  float64       
 5   disbursal_date                        138177 non-null  datetime64[ns]
 6   principal_amount                      138177 non-null  float64       
 7   total_associates                      138151 non-null  float64       
 8   female_associates                     138151 non-null  float64       
 9   max_associate_age                     138047 non-null  float64 

In [ ]:
df=df.dropna()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 134662 entries, 3 to 1519857
Data columns (total 21 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   industry_name                         134662 non-null  object        
 1   program_type                          134662 non-null  object        
 2   buyer_city                            134662 non-null  object        
 3   state_name                            134662 non-null  object        
 4   invoice_amount                        134662 non-null  float64       
 5   disbursal_date                        134662 non-null  datetime64[ns]
 6   principal_amount                      134662 non-null  float64       
 7   total_associates                      134662 non-null  float64       
 8   female_associates                     134662 non-null  float64       
 9   max_associate_age                     134662 non-null  float64 

In [ ]:
# Select all columns except 'disbursal_date'
categorical_columns = df.select_dtypes(include=['object']).columns.difference(['disbursal_date']).tolist()

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# One-hot encode the selected categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_columns])

# Create a DataFrame with the one-hot encoded features
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the original DataFrame with the one-hot encoded DataFrame
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns from the encoded DataFrame
df_encoded = df_encoded.drop(categorical_columns, axis=1)

# Update the original DataFrame with the encoded DataFrame
df = df_encoded


In [ ]:
import datetime
df['disbursal_date'] = pd.to_datetime(df['disbursal_date'])
target_date = datetime.datetime(2024, 3, 31)
train = df[df['disbursal_date'] <= target_date]
test = df[df['disbursal_date']>target_date]
train = train.drop(columns = ['disbursal_date'],axis=1)
test = test.drop(columns = ['disbursal_date'],axis=1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.shape,test.shape

((83941, 572), (50721, 572))

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)
X_train =train.drop('default', axis=1)  # Features
y_train = train['default']  # Target variable

# Instantiate ADASYN
adasyn = ADASYN()

# Apply ADASYN oversampling
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

# Concatenate X_resampled and y_resampled to get the oversampled dataset
train = pd.concat([X_resampled, y_resampled], axis=1)

In [ ]:

X_test =test.drop('default', axis=1)  # Features
y_test = test['default']  # Target variable

# Instantiate ADASYN
adasyn = ADASYN()

# Apply ADASYN oversampling
X_resampled1, y_resampled1 = adasyn.fit_resample(X_test, y_test)

# Concatenate X_resampled and y_resampled to get the oversampled dataset
test = pd.concat([X_resampled1, y_resampled1], axis=1)

In [ ]:
def encode(df):
    df_obj = df.select_dtypes(include=['object'])
    encoders = {}
    for col in df_obj.columns:
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])
        encoders[col] = encoder
    with open('LE_mdl_v1.pkl', 'wb') as f:
        pickle.dump(encoders, f)
    return df

In [ ]:
train.dropna(inplace=True)
test.dropna(inplace=True)
scaler = StandardScaler()
# Separate features and target variables for train and test datasets
x_train = train.drop(columns=[ 'repayment_days','default'],axis=1)
y_train = train['repayment_days']
Y_train = train['default']

x_val = test.drop(columns=[ 'repayment_days','default'])
y_val = test['repayment_days']
Y_val = test['default']

x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
print(x_val_scaled)

[[-0.4269319  -0.4184627   0.44781269 ... -0.48547696 -0.08374497
  -0.25556452]
 [-0.39833434 -0.38439276 -0.60597442 ... -0.48547696 -0.08374497
  -0.25556452]
 [ 0.53394602  0.72628723 -0.60597442 ... -0.48547696 -0.08374497
  -0.25556452]
 ...
 [ 1.18861557  1.50623326 -0.60597442 ... -0.48547696 -0.08374497
  -0.25556452]
 [ 0.76988611  1.00737642 -0.60597442 ... -0.48547696 -0.08374497
  -0.25556452]
 [-0.43904571 -0.43289458 -0.60597442 ... -0.48547696 -0.08374497
  -0.25556452]]


In [ ]:
model1 = LogisticRegression()
model2 = RandomForestClassifier()
model3 = DecisionTreeClassifier()

In [ ]:
import seaborn as sns
def model_train(model, x_train, y_train, x_test, y_test,x1):
    # Ensure y_test is 1-dimensional
    y_test = np.ravel(y_test)

    # Train the model
    model.fit(x_train, y_train)

    # Make predictions
    y_pred = model.predict(x_test)

    # Ensure y_pred is 1-dimensional
    y_pred = np.ravel(y_pred)

    # Calculate accuracy
    print('Accuracy Score:', accuracy_score(y_test, y_pred))

    # Print classification report
    print('Classification Report:')
    print(classification_report(y_test, y_pred))

    if hasattr(model, 'feature_importances_'):
        feature_importances = pd.Series(model.feature_importances_, index=x1.columns)
        top_feature_importances = feature_importances
        feature_importances.plot(kind='barh')
        plt.title('Top 10 Feature Importances')
        plt.xlabel('Relative Importance')
        plt.ylabel('Features')
        plt.show()
    top_feature_importances.to_csv('feature_importances.csv')

    # Compute confusion matrix
    unique_labels = np.unique(np.concatenate((y_test, y_pred)))
    conf_matrix = confusion_matrix(y_test, y_pred, labels=unique_labels)

    # Convert confusion matrix to DataFrame
    conf_matrix_df = pd.DataFrame(conf_matrix,
                                  index=unique_labels,
                                  columns=unique_labels)

    # Save the DataFrame to a CSV file
    conf_matrix_df.to_csv('confusion_matrix.csv')

    # Print confusion matrix
    print('Confusion Matrix:')
    print(conf_matrix_df)

    # Save the trained model
    with open(str(model.__class__.__name__) + '_mdl.pkl', 'wb') as f:
        pickle.dump(model, f)

    # # Save y_pred and y_test to the same CSV file side by side
    # x_test['y_test'] = y_test
    # x_test['y_pred'] = y_pred

    # # Save x_test, y_test, and y_pred to the same CSV file


In [ ]:
model_train(model2, x_train_scaled, y_train, x_val_scaled, y_val,x_train)



ValueError: Unknown label type: 'continuous'

In [ ]:
model_train(model3, x_train, Y_train, x_val, Y_val,x_train)

In [ ]:
# model3.fit(x_train_scaled, y_train)

#     # Make predictions
# y_pred = model3.predict(x_val_scaled)
# x_val['y_test'] = y_val
# x_val['y_pred'] = y_pred

#     # # Save x_test, y_test, and y_pred to the same CSV file
# x_val.to_csv('x1_y_pred_y_test.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm_model = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.

# Train the SVM model
svm_model.fit(x_train_scaled, Y_train)

# Predict on validation set
predictions = svm_model.predict(x_val_scaled)

# Evaluate the model (you might want to use different metrics based on your problem)
accuracy = svm_model.score(x_val_scaled, Y_val)
print("Accuracy:", accuracy)

In [ ]:
coef_matrix = pd.DataFrame(svm_model.coef_.T, index=x_train.columns)

# Save the coefficient matrix to a CSV file
coef_matrix.to_csv('svm_coefficients.csv')

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

svm_model1 = SVC(kernel='linear')  # You can choose different kernels like 'rbf', 'poly', etc.

# Train the SVM model
svm_model1.fit(x_train_scaled, y_train)

# Predict on validation set
predictions = svm_model1.predict(x_val_scaled)

# Evaluate the model (you might want to use different metrics based on your problem)
accuracy = svm_model.score(x_val_scaled, y_val)
print("Accuracy:", accuracy)

In [ ]:
print(predictions)

In [ ]:
feature_weights = pd.DataFrame({'Feature': x_train.columns, 'Weight': svm_model1.coef_[0]})
print("Feature Weights:")
print(feature_weights)

# Plotting decision boundary (works only for 2D or 3D data)
if len(x_train.columns) <= 3:
    if len(x_train.columns) == 2:
        # For 2D data
        x_min, x_max = x_train_scaled[:, 0].min() - 1, x_train_scaled[:, 0].max() + 1
        y_min, y_max = x_train_scaled[:, 1].min() - 1, x_train_scaled[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                             np.arange(y_min, y_max, 0.02))
        Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
        plt.scatter(x_train_scaled[:, 0], x_train_scaled[:, 1], c=Y_train, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        plt.title('Decision Boundary')
        plt.show()
    elif len(x_train.columns) == 3:
        # For 3D data
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        xx, yy = np.meshgrid(np.linspace(x_train_scaled[:, 0].min(), x_train_scaled[:, 0].max(), 50),
                             np.linspace(x_train_scaled[:, 1].min(), x_train_scaled[:, 1].max(), 50))
        Z = svm_model.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)
        ax.contour3D(xx, yy, Z, 50, cmap='coolwarm')
        ax.scatter(x_train_scaled[:, 0], x_train_scaled[:, 1], c=Y_train, cmap='coolwarm', s=20, edgecolors='k')
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.set_zlabel('Feature 3')
        ax.set_title('Decision Boundary')
        plt.show()
else:
    print("Cannot plot decision boundary for data with more than 3 features.")